In [22]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, roc_curve,auc
import warnings
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,precision_score,recall_score,roc_auc_score,accuracy_score,roc_curve

seed = 623
plt.style.use('ggplot')

In [23]:
# data是原始数据大表，包含所有特征
data1 = pd.read_csv('data_1.csv', index_col=0)
data2 = pd.read_csv('data_2.csv', index_col=0)
data = pd.merge(data1, data2, on='user')

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (2659,2672,2685,2698,2711,2724,2737,2750,2763,2776,2789) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [24]:
data.rename(columns = {'label_x':'label'}, inplace = True)
del data['label_y']

In [25]:
# 创造一个原始数据的备份，对数据进行操作
df = data.copy()

df_train = df[~df['label'].isnull()].copy()
df_test = df[df['label'].isnull()].copy()

df_train.shape, df_test.shape

((38225, 5244), (9557, 5244))

In [26]:
# data.info()

In [27]:
sample = df_train.sample(frac=0.1, random_state=623)
train_labels = sample.label
train_features = sample.drop(columns='label')

from feature_selector import FeatureSelector
fs = FeatureSelector(data=train_features, labels=train_labels)

In [28]:
fs.identify_all(selection_params = {'missing_threshold': 0.6,    
                                    'correlation_threshold': 0.9, 
                                    'task': 'classification',    
                                    'eval_metric': 'auc', 
                                    'cumulative_importance': 0.99})

164 features with greater than 0.60 missing values.

653 features with a single unique value.

3213 features with a correlation magnitude greater than 0.90.

Training Gradient Boosting Model

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[34]	valid_0's auc: 0.654865	valid_0's binary_logloss: 0.51639
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[52]	valid_0's auc: 0.727404	valid_0's binary_logloss: 0.489993
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[38]	valid_0's auc: 0.69403	valid_0's binary_logloss: 0.506796
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[40]	valid_0's auc: 0.696767	valid_0's binary_logloss: 0.498656
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[16]	valid_0's auc: 0.701056	valid_0's binary_logloss: 0.511525
Train

In [29]:
train_removed = fs.remove(methods = 'all', keep_one_hot=False)

['missing', 'single_unique', 'collinear', 'zero_importance', 'low_importance'] methods have been run

Removed 34232 features including one-hot features.


In [30]:
good_cols = train_removed.columns.tolist()
df_train_clear = df_train[good_cols]
df_test_clear = df_test[good_cols]

df_train_clear.insert(loc=0, column='label', value=df_train.label)
df_test_clear.insert(loc=0, column='label', value=df_test.label)

df_train_clear.shape, df_test_clear.shape

((38225, 746), (9557, 746))

In [31]:
df_clear = pd.concat([df_train_clear, df_test_clear], axis=0)
# df_clear['user'] = df.index
df_clear.reset_index(level=0, inplace=True)
df_clear.head()

,user,label,login_cnt_period1_x,province_x,city_x,sex_sex_x,sex_provider_x,sex_level_x,sex_verified_x,sex_regist_type_x,...,city_op2_cnt_mean,city_service1_cnt_mean,city_service1_amt_mean,city_agreement_total_mean,city_login_cnt_avg_mean,city_balance_avg_mean,city_balance2_mean,city_product1_amount_mean,city_product3_amount_mean,city_product7_cnt_mean
0,Train_00000,0.0,0.002782,17,45,category 0_category 0,category 0_category 0,category 0_category 1,category 0_category 0,category 0_category 1,...,0.023251,0.000873,0.000930,0.329782,0.024373,0.456977,0.394828,0.091954,0.086207,0.050836
1,Train_00001,1.0,0.006121,12,204,category 0_category 0,category 0_category 0,category 0_category 2,category 0_category 0,category 0_category 1,...,0.022878,0.000984,0.000758,0.316146,0.023442,0.435602,0.382383,0.064767,0.016839,0.060533
2,Train_00002,0.0,0.015597,5,331,category 0_category 0,category 0_category 0,category 0_category 2,category 0_category 0,category 0_category 7,...,0.023444,0.002224,0.001742,0.363854,0.020695,0.455987,0.341534,0.095847,0.102236,0.055430
3,Train_00005,0.0,0.011220,21,326,category 0_category 0,category 0_category 0,category 0_category 2,category 0_category 0,category 0_category 1,...,0.023927,0.003038,0.002174,0.337236,0.025890,0.474667,0.403333,0.066667,0.026667,0.056911
4,Train_00006,0.0,0.087933,13,106,category 0_category 0,category 0_category 0,category 0_category 2,category 0_category 0,category 0_category 7,...,0.024291,0.004410,0.006652,0.354192,0.020660,0.527459,0.434631,0.081967,0.024590,0.057943


In [32]:
df_clear.shape

(47782, 747)

In [33]:
df_clear.to_csv('data_clear.csv')